# SWR

A flexible framework for safe withdrawal rate experiments.

Framework can generalize to
- Any generator of asset returns
- Any asset allocation strategy based on age, returns etc.
- Any utility function to evaluate suitability of strategy (e.g. total spending, certainty equivalent spending)
- Support a survival table 
- Any optimizer to find optimal parameters for a given withdrawal framework and market simulation


In [1]:
import pytest
import numpy as np
import pandas as pd
import pandas_datareader as pdr

from SWRsimulation import SWRsimulation

In [2]:
# load from pickle
RETURN_FILE = 'histretSP'
def load_returns():
    return pd.read_pickle('%s.pickle' % RETURN_FILE)

download_df = load_returns()
return_df = download_df.iloc[:, [0, 3, 12]]
return_df.columns=['stocks', 'bonds', 'cpi']
return_df

,stocks,bonds,cpi
Year,,,
1928,0.438112,0.032196,-0.011522
1929,-0.082979,0.030179,0.000000
1930,-0.251236,0.005398,-0.026712
1931,-0.438375,-0.156808,-0.089321
1932,-0.086424,0.235896,-0.103014
...,...,...,...
2016,0.117731,0.103651,0.012616
2017,0.216055,0.097239,0.021301
2018,-0.042269,-0.027626,0.024426


In [3]:
# should adjust CPI to year-ending also but leave it for now
real_return_df = return_df.copy()
# real_return_df.loc[1948:, 'cpi'] = cpi_test['cpi_fred']
# adjust returns for inflation
real_return_df['stocks'] = (1 + real_return_df['stocks']) / (1 + real_return_df['cpi']) - 1
real_return_df['bonds'] = (1 + real_return_df['bonds']) / (1 + real_return_df['cpi']) - 1
real_return_df.drop('cpi', axis=1, inplace=True)
real_return_df

,stocks,bonds
Year,,
1928,0.454874,0.044227
1929,-0.082979,0.030179
1930,-0.230686,0.032991
1931,-0.383290,-0.074106
1932,0.018495,0.377832
...,...,...
2016,0.103805,0.089901
2017,0.190692,0.074354
2018,-0.065104,-0.050811


In [4]:
# zero returns, zero spending (just check shape)
RETURN = 0.0
# spending
FIXED = 0
VARIABLE = 0.0
NYEARS = 30

s = SWRsimulation.SWRsimulation({
    'simulation': {'returns_df': pd.DataFrame(index=range(1928, 2021), 
                                              data={'stocks': RETURN, 
                                                    'bonds': RETURN}),
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE},
    'evaluation': {}, # no args, default = number of years to exhaustion
})

print(s)

# just simulate 1st year in trials
z = s.simulate_trial(next(s.simulation['trials']))
assert len(z) == 30
assert(z.index[0]) == 1928, "start year == 1928"
assert(z.index[-1]) == 1957, "end year == 1957"

z

Simulation:
{'n_asset_years': 93,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':       stocks  bonds
1928     0.0    0.0
1929     0.0    0.0
1930     0.0    0.0
1931     0.0    0.0
1932     0.0    0.0
...      ...    ...
2016     0.0    0.0
2017     0.0    0.0
2018     0.0    0.0
2019     0.0    0.0
2020     0.0    0.0

[93 rows x 2 columns],
 'trials': <generator object SWRsimulation.historical_trials at 0x7fa07818dbd0>}

Allocation:
{}

Withdrawal:
{'fixed_pct': 0, 'variable_pct': 0.0}


,start_port,port_return,before_spend,spend,end_port,alloc_0,alloc_1
1928,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1929,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1930,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1931,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1932,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1933,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1934,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1935,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1936,100.0,0.0,100.0,0.0,100.0,0.5,0.5
1937,100.0,0.0,100.0,0.0,100.0,0.5,0.5


In [5]:
# zero returns, spend 2% of starting portfolio per year, check ending value declines to 0.4
RETURN = 0.0
# spending
FIXED = 2.0
VARIABLE = 0.00
NYEARS = 30

returns_df = pd.DataFrame(index=range(1928, 2021), data={'stocks': RETURN, 'bonds': RETURN})

s = SWRsimulation.SWRsimulation({
    'simulation': {'returns_df': returns_df,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE},
    'evaluation': {}, # no args, default = number of years to exhaustion
})

print(s)

z = s.simulate_trial(next(s.simulation['trials']))

assert(z['start_port'].iloc[0]) == 100, "start port value == 100"
assert(z['end_port'].iloc[-1]) == 40, "ending port value == 40"

z

Simulation:
{'n_asset_years': 93,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':       stocks  bonds
1928     0.0    0.0
1929     0.0    0.0
1930     0.0    0.0
1931     0.0    0.0
1932     0.0    0.0
...      ...    ...
2016     0.0    0.0
2017     0.0    0.0
2018     0.0    0.0
2019     0.0    0.0
2020     0.0    0.0

[93 rows x 2 columns],
 'trials': <generator object SWRsimulation.historical_trials at 0x7fa0988c1ad0>}

Allocation:
{}

Withdrawal:
{'fixed_pct': 2.0, 'variable_pct': 0.0}


,start_port,port_return,before_spend,spend,end_port,alloc_0,alloc_1
1928,100.0,0.0,100.0,2.0,98.0,0.5,0.5
1929,98.0,0.0,98.0,2.0,96.0,0.5,0.5
1930,96.0,0.0,96.0,2.0,94.0,0.5,0.5
1931,94.0,0.0,94.0,2.0,92.0,0.5,0.5
1932,92.0,0.0,92.0,2.0,90.0,0.5,0.5
1933,90.0,0.0,90.0,2.0,88.0,0.5,0.5
1934,88.0,0.0,88.0,2.0,86.0,0.5,0.5
1935,86.0,0.0,86.0,2.0,84.0,0.5,0.5
1936,84.0,0.0,84.0,2.0,82.0,0.5,0.5
1937,82.0,0.0,82.0,2.0,80.0,0.5,0.5


In [6]:
# zero returns, spend 2% of current portfolio per year, check ending value declines to 0.98 ** 30
RETURN = 0.0
FIXED = 0
VARIABLE = 2.0
NYEARS = 30

s = SWRsimulation.SWRsimulation({
    'simulation': {'returns_df': pd.DataFrame(index=range(1928, 2021), data={'stocks': RETURN, 'bonds': RETURN}),
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE},
    'evaluation': {}, # no args, default = number of years to exhaustion
})

print(s)

z = s.simulate_trial(next(s.simulation['trials']))

assert(z['start_port'].iloc[0]) == 100, "start port value == 100"
assert z['end_port'].iloc[-1] == pytest.approx(100 * ((1 - VARIABLE/100) ** NYEARS), 0.000001)
z

Simulation:
{'n_asset_years': 93,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':       stocks  bonds
1928     0.0    0.0
1929     0.0    0.0
1930     0.0    0.0
1931     0.0    0.0
1932     0.0    0.0
...      ...    ...
2016     0.0    0.0
2017     0.0    0.0
2018     0.0    0.0
2019     0.0    0.0
2020     0.0    0.0

[93 rows x 2 columns],
 'trials': <generator object SWRsimulation.historical_trials at 0x7fa04818a4d0>}

Allocation:
{}

Withdrawal:
{'fixed_pct': 0, 'variable_pct': 2.0}


,start_port,port_return,before_spend,spend,end_port,alloc_0,alloc_1
1928,100.000000,0.0,100.000000,2.000000,98.000000,0.5,0.5
1929,98.000000,0.0,98.000000,1.960000,96.040000,0.5,0.5
1930,96.040000,0.0,96.040000,1.920800,94.119200,0.5,0.5
1931,94.119200,0.0,94.119200,1.882384,92.236816,0.5,0.5
1932,92.236816,0.0,92.236816,1.844736,90.392080,0.5,0.5
1933,90.392080,0.0,90.392080,1.807842,88.584238,0.5,0.5
1934,88.584238,0.0,88.584238,1.771685,86.812553,0.5,0.5
1935,86.812553,0.0,86.812553,1.736251,85.076302,0.5,0.5
1936,85.076302,0.0,85.076302,1.701526,83.374776,0.5,0.5
1937,83.374776,0.0,83.374776,1.667496,81.707281,0.5,0.5


In [7]:
# 4% real return, spend fixed 4% of starting, assert ending value unchanged
RETURN = 0.04
FIXED = 4 
VARIABLE = 0.0
NYEARS = 30

returns_df = pd.DataFrame(index=range(1928, 2021), data={'stocks': RETURN, 'bonds': RETURN})

s = SWRsimulation.SWRsimulation({
    'simulation': {'returns_df': returns_df,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE},
    'evaluation': {}, # no args, default = number of years to exhaustion
})

print(s)

z = s.simulate_trial(next(s.simulation['trials']))

assert(z['start_port'].iloc[0]) == 100, "start port value == 100"
assert(z['end_port'].iloc[-1]) == 100, "end port value correct"
z


Simulation:
{'n_asset_years': 93,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':       stocks  bonds
1928    0.04   0.04
1929    0.04   0.04
1930    0.04   0.04
1931    0.04   0.04
1932    0.04   0.04
...      ...    ...
2016    0.04   0.04
2017    0.04   0.04
2018    0.04   0.04
2019    0.04   0.04
2020    0.04   0.04

[93 rows x 2 columns],
 'trials': <generator object SWRsimulation.historical_trials at 0x7fa04818aa50>}

Allocation:
{}

Withdrawal:
{'fixed_pct': 4, 'variable_pct': 0.0}


,start_port,port_return,before_spend,spend,end_port,alloc_0,alloc_1
1928,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1929,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1930,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1931,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1932,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1933,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1934,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1935,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1936,100.0,0.04,104.0,4.0,100.0,0.5,0.5
1937,100.0,0.04,104.0,4.0,100.0,0.5,0.5


In [8]:
# return 0.02% variable spending 0.02/1.02, check final value unchanged
RETURN = 0.02
FIXED = 0.0
VARIABLE = 0.02/1.02*100
NYEARS = 30

returns_df = pd.DataFrame(index=range(1928, 2021), data={'stocks': RETURN, 'bonds': RETURN})

s = SWRsimulation.SWRsimulation({
    'simulation': {'returns_df': returns_df,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE},
    'evaluation': {}, # no args, default = number of years to exhaustion
})

print(s)

z = s.simulate_trial(next(s.simulation['trials']))

assert (z['start_port'].iloc[0]) == 100, "start port value == 100"
assert (z['end_port'].iloc[-1]) == 100, "end port value correct"
z


Simulation:
{'n_asset_years': 93,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':       stocks  bonds
1928    0.02   0.02
1929    0.02   0.02
1930    0.02   0.02
1931    0.02   0.02
1932    0.02   0.02
...      ...    ...
2016    0.02   0.02
2017    0.02   0.02
2018    0.02   0.02
2019    0.02   0.02
2020    0.02   0.02

[93 rows x 2 columns],
 'trials': <generator object SWRsimulation.historical_trials at 0x7fa0989bc150>}

Allocation:
{}

Withdrawal:
{'fixed_pct': 0.0, 'variable_pct': 1.9607843137254901}


,start_port,port_return,before_spend,spend,end_port,alloc_0,alloc_1
1928,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1929,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1930,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1931,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1932,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1933,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1934,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1935,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1936,100.0,0.02,102.0,2.0,100.0,0.5,0.5
1937,100.0,0.02,102.0,2.0,100.0,0.5,0.5


In [9]:
# check values per appendix of Bengen paper https://www.retailinvestor.org/pdf/Bengen1.pdf
# nominal return 10% for stocks, 5% for bonds
# inflation 3%
# fixed spending of 4% of orig port
STOCK_RETURN = (1.1 / 1.03) - 1
BOND_RETURN = (1.05 / 1.03) - 1
VARIABLE = 0.0
FIXED = 4.0
NYEARS = 30

returns_df = pd.DataFrame(index=range(1928, 2021), data={'stocks': STOCK_RETURN, 'bonds': BOND_RETURN})

s = SWRsimulation.SWRsimulation({
    'simulation': {'returns_df': returns_df,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE},
    'evaluation': {}, # no args, default = number of years to exhaustion
})

print(s)

z = s.simulate_trial(next(s.simulation['trials']))

# match figures in appendix
# example uses nominal vals with 3% inflation, we use real vals
assert z.iloc[0]['before_spend'] * 1.03 == pytest.approx(107.5, 0.000001)
assert z.iloc[0]['spend'] * 1.03 == 4.12, "spend does not match Bengen"
assert z.iloc[0]['end_port'] * 1.03 == pytest.approx(103.38, 0.000001), "ending port does not match Bengen"

z

Simulation:
{'n_asset_years': 93,
 'n_assets': 2,
 'n_ret_years': 30,
 'returns_df':         stocks     bonds
1928  0.067961  0.019417
1929  0.067961  0.019417
1930  0.067961  0.019417
1931  0.067961  0.019417
1932  0.067961  0.019417
...        ...       ...
2016  0.067961  0.019417
2017  0.067961  0.019417
2018  0.067961  0.019417
2019  0.067961  0.019417
2020  0.067961  0.019417

[93 rows x 2 columns],
 'trials': <generator object SWRsimulation.historical_trials at 0x7fa04818a1d0>}

Allocation:
{}

Withdrawal:
{'fixed_pct': 4.0, 'variable_pct': 0.0}


,start_port,port_return,before_spend,spend,end_port,alloc_0,alloc_1
1928,100.000000,0.043689,104.368932,4.0,100.368932,0.5,0.5
1929,100.368932,0.043689,104.753982,4.0,100.753982,0.5,0.5
1930,100.753982,0.043689,105.155855,4.0,101.155855,0.5,0.5
1931,101.155855,0.043689,105.575286,4.0,101.575286,0.5,0.5
1932,101.575286,0.043689,106.013041,4.0,102.013041,0.5,0.5
1933,102.013041,0.043689,106.469922,4.0,102.469922,0.5,0.5
1934,102.469922,0.043689,106.946763,4.0,102.946763,0.5,0.5
1935,102.946763,0.043689,107.444437,4.0,103.444437,0.5,0.5
1936,103.444437,0.043689,107.963854,4.0,103.963854,0.5,0.5
1937,103.963854,0.043689,108.505964,4.0,104.505964,0.5,0.5


In [ ]:
VARIABLE = 0.0
FIXED = 5.0
NYEARS = 30

s = SWRsimulation.SWRsimulation({
    'simulation': {'returns_df': real_return_df,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE},
    'evaluation': {}, # no args, default = number of years to exhaustion
})

print(s)

z = s.simulate()

In [ ]:
years_survived = pd.DataFrame(data={'nyears': [30 - len(np.where(y['spend']==0.0)[0]) for y in z]},
                              index=range(1928,1992)).reset_index()

px.bar(years_survived, x="index", y="nyears", color="nyears",
              hover_name="index", color_continuous_scale="spectral")


In [ ]:
portval_df = pd.DataFrame(data=np.hstack([(np.ones(64).reshape(64, 1) * 100), np.array([y['end_port'].values for y in z])])) \
    .transpose()
portval_df.columns=range(1928,1992)
portval_df['mean'] = portval_df.mean(axis=1)
portval_df

col_list = list(portval_df.columns)
portval_df.reset_index(inplace=True)
portval_melt = pd.melt(portval_df, id_vars=['index'], value_vars=col_list)
portval_melt.columns=['ret_year', 'start_year', 'portval']
portval_melt

In [ ]:
portval_df

In [ ]:
fig = go.Figure()
for year in range(1928,1992):
    
    fig.add_trace(go.Scatter(x=portval_df['index'], 
                             y=portval_df[year],
                             mode='lines',
                             name=str(year),
                             line={'width': 1},
                            ),
                 )

fig.add_trace(go.Scatter(x=portval_df['index'], 
                         y=portval_df['mean'],
                         mode='lines',
                         name='Mean',
                         line={'width': 3, 'color': 'black'},
                        ),
             )
    
fig.update_layout(showlegend=False,
                  plot_bgcolor="white",
                  title="Retirement Outcomes, 1928-1991",
                  xaxis=dict(title="Retirement Year", linecolor='black', mirror=True, ticks='inside',),
                  yaxis=dict(title="Portfolio Value", linecolor='black', mirror=True, ticks='inside'),
                 )
fig.show()


In [ ]:
fig = px.line(portval_melt, x="ret_year", y="portval", color="start_year",
              hover_name="start_year")
fig.show()

In [ ]:
VARIABLE = 0.0
FIXED = 4.0
NYEARS = 30
NTRIALS = 10000

s = SWRsimulation.SWRsimulation({
    'simulation': {'returns_df': real_return_df,
                   'montecarlo': 10000,
                   'montecarlo_replacement': False,
                   'n_ret_years': NYEARS,
                  },
    'allocation': {},  # no args, default equal weight
    'withdrawal': {'fixed_pct': FIXED,
                   'variable_pct': VARIABLE},
    'evaluation': {}, # no args, default = number of years to exhaustion
})

print(s)

z = s.simulate(do_eval=True, return_both=False)

In [ ]:
c, bins = np.histogram(z, bins=list(range(31)))
pct_exhausted = np.sum(c[:29])/np.sum(c) * 100
print ("%.2f%% of portfolios exhausted before final year" % pct_exhausted)
bins += 1
fig = go.Figure([go.Bar(x=bins, y=c)])
fig.update_layout(showlegend=False,
                  plot_bgcolor="white",
                  title="Histogram of years to exhaustion",
                  xaxis=dict(title="Retirement Year", 
                             linecolor='black', mirror=True, ticks='inside',),
                  yaxis=dict(title="Number of Portfolios Exhausted (log scale)", 
                             linecolor='black', mirror=True, ticks='inside',
                             type="log"),
                 )

fig

In [ ]:
# .analyze() ... run the 2 plots
# analyze_plotly, analyze
# use montecarlo to find a bad example with 4%
# use a fixture to test the eval code, clean up tests

# function to return ce value based on strategy
# ce value of a stream
# ce value of many ce streams
# run bayesian optimizaton, optuna etc.
# ce value of a set of streams over a mortality curve
# try convex optimization
# go through the scikit optimizers and see if there are other global optimizers that might work eg simulated annealing
# accept an arbitrary schedule, utility function, optimizer


# certainty_equivalent_mortality(k_list, mortality_table)
# calculate probability of being alive in years 1...k
# so now you have k lists with associated probabilities
#    certainty_adjust cash flow streams
#    not just full cash flow but range(nyears) cash flows 
#    then certainty adjust over the probability distribution of each sequence of cash flows
    